In [1]:
import scipy.misc
import random
import os
import tensorflow as tf
from tensorflow.core.protobuf import saver_pb2
import scipy
import cv2
from subprocess import call

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Load data

In [8]:
xs = []
ys = []

#points to the end of the last batch
train_batch_pointer = 0
val_batch_pointer = 0

#read data.txt
with open("driving_dataset\\driving_dataset\\data.txt") as f:
    for line in f:
        xs.append("driving_dataset\\driving_dataset\\" + line.split()[0])
        #the paper by Nvidia uses the inverse of the turning radius,
        #but steering wheel angle is proportional to the inverse of turning radius
        #so the steering wheel angle in radians is used as the output
        ys.append(float(line.split()[1]) * scipy.pi / 180)

#get number of images
num_images = len(xs)

#shuffle list of images
#c = list(zip(xs, ys))
#random.shuffle(c)
#xs, ys = zip(*c)

train_xs = xs[:int(len(xs) * 0.7)]
train_ys = ys[:int(len(xs) * 0.7)]

val_xs = xs[-int(len(xs) * 0.3):]
val_ys = ys[-int(len(xs) * 0.3):]

num_train_images = len(train_xs)
num_val_images = len(val_xs)

def LoadTrainBatch(batch_size):
    global train_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
        x_out.append(scipy.misc.imresize(scipy.misc.imread(train_xs[(train_batch_pointer + i) % num_train_images])[-150:], [66, 200]) / 255.0)
        y_out.append([train_ys[(train_batch_pointer + i) % num_train_images]])
    train_batch_pointer += batch_size
    return x_out, y_out

def LoadValBatch(batch_size):
    global val_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
        x_out.append(scipy.misc.imresize(scipy.misc.imread(val_xs[(val_batch_pointer + i) % num_val_images])[-150:], [66, 200]) / 255.0)
        y_out.append([val_ys[(val_batch_pointer + i) % num_val_images]])
    val_batch_pointer += batch_size
    return x_out, y_out

## Model

In [3]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W, stride):
    return tf.nn.conv2d(x, W, strides=[1, stride, stride, 1], padding='VALID')

x = tf.placeholder(tf.float32, shape=[None, 66, 200, 3])
y_ = tf.placeholder(tf.float32, shape=[None, 1])

x_image = x

#first convolutional layer
W_conv1 = weight_variable([5, 5, 3, 24])
b_conv1 = bias_variable([24])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1, 2) + b_conv1)

#second convolutional layer
W_conv2 = weight_variable([5, 5, 24, 36])
b_conv2 = bias_variable([36])

h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2, 2) + b_conv2)

#third convolutional layer
W_conv3 = weight_variable([5, 5, 36, 48])
b_conv3 = bias_variable([48])

h_conv3 = tf.nn.relu(conv2d(h_conv2, W_conv3, 2) + b_conv3)

#fourth convolutional layer
W_conv4 = weight_variable([3, 3, 48, 64])
b_conv4 = bias_variable([64])

h_conv4 = tf.nn.relu(conv2d(h_conv3, W_conv4, 1) + b_conv4)

#fifth convolutional layer
W_conv5 = weight_variable([3, 3, 64, 64])
b_conv5 = bias_variable([64])

h_conv5 = tf.nn.relu(conv2d(h_conv4, W_conv5, 1) + b_conv5)

#FCL 1
W_fc1 = weight_variable([1152, 1164])
b_fc1 = bias_variable([1164])

h_conv5_flat = tf.reshape(h_conv5, [-1, 1152])
h_fc1 = tf.nn.relu(tf.matmul(h_conv5_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

#FCL 2
W_fc2 = weight_variable([1164, 100])
b_fc2 = bias_variable([100])

h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

#FCL 3
W_fc3 = weight_variable([100, 50])
b_fc3 = bias_variable([50])

h_fc3 = tf.nn.relu(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)

h_fc3_drop = tf.nn.dropout(h_fc3, keep_prob)

#FCL 3
W_fc4 = weight_variable([50, 10])
b_fc4 = bias_variable([10])

h_fc4 = tf.nn.relu(tf.matmul(h_fc3_drop, W_fc4) + b_fc4)

h_fc4_drop = tf.nn.dropout(h_fc4, keep_prob)

#Output
W_fc5 = weight_variable([10, 1])
b_fc5 = bias_variable([1])

y = tf.matmul(h_fc4_drop, W_fc5) + b_fc5

## Training

In [19]:
LOGDIR = 'save'

sess = tf.InteractiveSession()

L2NormConst = 0.001

train_vars = tf.trainable_variables()

loss = tf.reduce_mean(tf.square(tf.subtract(y_, y))) + tf.add_n([tf.nn.l2_loss(v) for v in train_vars]) * L2NormConst
train_step = tf.train.AdamOptimizer(1e-3).minimize(loss)
sess.run(tf.initialize_all_variables())

# create a summary to monitor cost tensor
tf.summary.scalar("loss", loss)
# merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

saver = tf.train.Saver(write_version = saver_pb2.SaverDef.V1)

# op to write logs to Tensorboard
logs_path = 'logs'
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

epochs = 30
batch_size = 100

# train over the dataset about 30 times
for epoch in range(epochs):
    for i in range(int(num_images/batch_size)):
        xs, ys = LoadTrainBatch(batch_size)
        train_step.run(feed_dict={x: xs, y_: ys, keep_prob: 0.5})
        if i % 10 == 0:
            xs, ys = LoadValBatch(batch_size)
            loss_value = loss.eval(feed_dict={x:xs,y_: ys, keep_prob: 1.0})
            print("Epoch: %d, Step: %d, Loss: %g" % (epoch, epoch * batch_size + i, loss_value))

    # write logs at every iteration
    summary = merged_summary_op.eval(feed_dict={x:xs, y_: ys, keep_prob: 1.0})
    summary_writer.add_summary(summary, epoch * num_images/batch_size + i)

    if i % batch_size == 0:
        if not os.path.exists(LOGDIR):
            os.makedirs(LOGDIR)
        checkpoint_path = os.path.join(LOGDIR, "model.ckpt")
        filename = saver.save(sess, checkpoint_path)
        print("Model saved in file: %s" % filename)

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\client\session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:49: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imag

Epoch: 0, Step: 0, Loss: 6.48709
Epoch: 0, Step: 10, Loss: 5.68935
Epoch: 0, Step: 20, Loss: 5.35863
Epoch: 0, Step: 30, Loss: 5.04059
Epoch: 0, Step: 40, Loss: 5.00071
Epoch: 0, Step: 50, Loss: 4.42687
Epoch: 0, Step: 60, Loss: 4.34338
Epoch: 0, Step: 70, Loss: 4.31678
Epoch: 0, Step: 80, Loss: 3.98552
Epoch: 0, Step: 90, Loss: 3.61126
Epoch: 0, Step: 100, Loss: 3.43652
Epoch: 0, Step: 110, Loss: 3.25853
Epoch: 0, Step: 120, Loss: 3.16537
Epoch: 0, Step: 130, Loss: 3.31007
Epoch: 0, Step: 140, Loss: 3.66675
Epoch: 0, Step: 150, Loss: 3.08108
Epoch: 0, Step: 160, Loss: 3.58149
Epoch: 0, Step: 170, Loss: 2.86535
Epoch: 0, Step: 180, Loss: 3.09275
Epoch: 0, Step: 190, Loss: 2.75313
Epoch: 0, Step: 200, Loss: 2.34884
Epoch: 0, Step: 210, Loss: 2.27096
Epoch: 0, Step: 220, Loss: 2.24781
Epoch: 0, Step: 230, Loss: 2.2701
Epoch: 0, Step: 240, Loss: 3.08496
Epoch: 0, Step: 250, Loss: 2.1961
Epoch: 0, Step: 260, Loss: 4.18971
Epoch: 0, Step: 270, Loss: 2.45325
Epoch: 0, Step: 280, Loss: 2.9656

Epoch: 5, Step: 500, Loss: 0.0656649
Epoch: 5, Step: 510, Loss: 0.0667936
Epoch: 5, Step: 520, Loss: 0.0654356
Epoch: 5, Step: 530, Loss: 0.0652032
Epoch: 5, Step: 540, Loss: 0.064546
Epoch: 5, Step: 550, Loss: 0.0639104
Epoch: 5, Step: 560, Loss: 0.0621424
Epoch: 5, Step: 570, Loss: 0.0623232
Epoch: 5, Step: 580, Loss: 0.0706278
Epoch: 5, Step: 590, Loss: 0.117084
Epoch: 5, Step: 600, Loss: 2.82365
Epoch: 5, Step: 610, Loss: 2.81309
Epoch: 5, Step: 620, Loss: 0.0598923
Epoch: 5, Step: 630, Loss: 0.0575613
Epoch: 5, Step: 640, Loss: 0.114135
Epoch: 5, Step: 650, Loss: 0.120505
Epoch: 5, Step: 660, Loss: 0.104006
Epoch: 5, Step: 670, Loss: 0.106356
Epoch: 5, Step: 680, Loss: 0.166622
Epoch: 5, Step: 690, Loss: 0.0557531
Epoch: 5, Step: 700, Loss: 0.0801327
Epoch: 5, Step: 710, Loss: 0.0743418
Epoch: 5, Step: 720, Loss: 0.0670347
Epoch: 5, Step: 730, Loss: 0.0589276
Epoch: 5, Step: 740, Loss: 0.151107
Epoch: 5, Step: 750, Loss: 0.142465
Epoch: 5, Step: 760, Loss: 0.136455
Epoch: 5, Step:

Epoch: 9, Step: 1290, Loss: 0.0431034
Epoch: 9, Step: 1300, Loss: 0.287573
Epoch: 9, Step: 1310, Loss: 0.615298
Epoch: 9, Step: 1320, Loss: 0.278454
Epoch: 9, Step: 1330, Loss: 0.0952627
Epoch: 9, Step: 1340, Loss: 0.0487644
Epoch: 9, Step: 1350, Loss: 0.0171204
Epoch: 10, Step: 1000, Loss: 0.116768
Epoch: 10, Step: 1010, Loss: 0.832726
Epoch: 10, Step: 1020, Loss: 1.48255
Epoch: 10, Step: 1030, Loss: 0.231269
Epoch: 10, Step: 1040, Loss: 0.122512
Epoch: 10, Step: 1050, Loss: 0.00588254
Epoch: 10, Step: 1060, Loss: 0.0114598
Epoch: 10, Step: 1070, Loss: 0.00809517
Epoch: 10, Step: 1080, Loss: 0.0188004
Epoch: 10, Step: 1090, Loss: 0.0440887
Epoch: 10, Step: 1100, Loss: 0.0486591
Epoch: 10, Step: 1110, Loss: 0.0104848
Epoch: 10, Step: 1120, Loss: 0.123909
Epoch: 10, Step: 1130, Loss: 0.396189
Epoch: 10, Step: 1140, Loss: 0.100567
Epoch: 10, Step: 1150, Loss: 0.179105
Epoch: 10, Step: 1160, Loss: 0.0157208
Epoch: 10, Step: 1170, Loss: 0.0168693
Epoch: 10, Step: 1180, Loss: 0.0172079
Epoc

Epoch: 14, Step: 1600, Loss: 0.037691
Epoch: 14, Step: 1610, Loss: 0.132931
Epoch: 14, Step: 1620, Loss: 0.0740068
Epoch: 14, Step: 1630, Loss: 0.0326987
Epoch: 14, Step: 1640, Loss: 0.0197849
Epoch: 14, Step: 1650, Loss: 0.0199298
Epoch: 14, Step: 1660, Loss: 0.0195662
Epoch: 14, Step: 1670, Loss: 0.0204165
Epoch: 14, Step: 1680, Loss: 0.020285
Epoch: 14, Step: 1690, Loss: 0.0196597
Epoch: 14, Step: 1700, Loss: 0.0196236
Epoch: 14, Step: 1710, Loss: 0.0345845
Epoch: 14, Step: 1720, Loss: 0.0346857
Epoch: 14, Step: 1730, Loss: 0.0151231
Epoch: 14, Step: 1740, Loss: 0.0161122
Epoch: 14, Step: 1750, Loss: 0.0760675
Epoch: 14, Step: 1760, Loss: 0.0348738
Epoch: 14, Step: 1770, Loss: 0.316817
Epoch: 14, Step: 1780, Loss: 0.0104829
Epoch: 14, Step: 1790, Loss: 0.0832349
Epoch: 14, Step: 1800, Loss: 0.0909454
Epoch: 14, Step: 1810, Loss: 0.3947
Epoch: 14, Step: 1820, Loss: 0.0530859
Epoch: 14, Step: 1830, Loss: 0.12573
Epoch: 14, Step: 1840, Loss: 0.290653
Epoch: 14, Step: 1850, Loss: 0.2173

Epoch: 19, Step: 1920, Loss: 0.00879553
Epoch: 19, Step: 1930, Loss: 0.0259869
Epoch: 19, Step: 1940, Loss: 0.0644092
Epoch: 19, Step: 1950, Loss: 0.0125833
Epoch: 19, Step: 1960, Loss: 0.0159655
Epoch: 19, Step: 1970, Loss: 0.119493
Epoch: 19, Step: 1980, Loss: 0.404928
Epoch: 19, Step: 1990, Loss: 0.21952
Epoch: 19, Step: 2000, Loss: 0.016232
Epoch: 19, Step: 2010, Loss: 0.010584
Epoch: 19, Step: 2020, Loss: 0.0111177
Epoch: 19, Step: 2030, Loss: 0.0117394
Epoch: 19, Step: 2040, Loss: 0.0109371
Epoch: 19, Step: 2050, Loss: 0.354049
Epoch: 19, Step: 2060, Loss: 5.05084
Epoch: 19, Step: 2070, Loss: 0.0601987
Epoch: 19, Step: 2080, Loss: 0.0111035
Epoch: 19, Step: 2090, Loss: 0.00573559
Epoch: 19, Step: 2100, Loss: 0.0121098
Epoch: 19, Step: 2110, Loss: 0.00890762
Epoch: 19, Step: 2120, Loss: 0.00691958
Epoch: 19, Step: 2130, Loss: 0.00216821
Epoch: 19, Step: 2140, Loss: 0.00217918
Epoch: 19, Step: 2150, Loss: 0.00194713
Epoch: 19, Step: 2160, Loss: 0.00156418
Epoch: 19, Step: 2170, Los

Epoch: 23, Step: 2590, Loss: 0.0304976
Epoch: 23, Step: 2600, Loss: 0.0944533
Epoch: 23, Step: 2610, Loss: 0.162286
Epoch: 23, Step: 2620, Loss: 0.151945
Epoch: 23, Step: 2630, Loss: 0.0283425
Epoch: 23, Step: 2640, Loss: 0.118499
Epoch: 23, Step: 2650, Loss: 0.182843
Epoch: 23, Step: 2660, Loss: 0.335238
Epoch: 23, Step: 2670, Loss: 0.0162353
Epoch: 23, Step: 2680, Loss: 0.21147
Epoch: 23, Step: 2690, Loss: 0.358183
Epoch: 23, Step: 2700, Loss: 0.0450004
Epoch: 23, Step: 2710, Loss: 0.0507612
Epoch: 23, Step: 2720, Loss: 0.00463071
Epoch: 23, Step: 2730, Loss: 0.0329004
Epoch: 23, Step: 2740, Loss: 0.0243202
Epoch: 23, Step: 2750, Loss: 0.700124
Epoch: 24, Step: 2400, Loss: 0.323714
Epoch: 24, Step: 2410, Loss: 0.663005
Epoch: 24, Step: 2420, Loss: 0.866983
Epoch: 24, Step: 2430, Loss: 0.127965
Epoch: 24, Step: 2440, Loss: 0.641043
Epoch: 24, Step: 2450, Loss: 0.158404
Epoch: 24, Step: 2460, Loss: 0.00793988
Epoch: 24, Step: 2470, Loss: 0.0269939
Epoch: 24, Step: 2480, Loss: 0.142827


Epoch: 28, Step: 2910, Loss: 2.97748
Epoch: 28, Step: 2920, Loss: 0.0133024
Epoch: 28, Step: 2930, Loss: 0.00464066
Epoch: 28, Step: 2940, Loss: 0.00469412
Epoch: 28, Step: 2950, Loss: 0.0124386
Epoch: 28, Step: 2960, Loss: 0.00390123
Epoch: 28, Step: 2970, Loss: 0.00615117
Epoch: 28, Step: 2980, Loss: 0.00177073
Epoch: 28, Step: 2990, Loss: 0.0024584
Epoch: 28, Step: 3000, Loss: 0.00316444
Epoch: 28, Step: 3010, Loss: 0.000855561
Epoch: 28, Step: 3020, Loss: 0.00926342
Epoch: 28, Step: 3030, Loss: 0.0426466
Epoch: 28, Step: 3040, Loss: 0.0392659
Epoch: 28, Step: 3050, Loss: 0.00226487
Epoch: 28, Step: 3060, Loss: 0.000672458
Epoch: 28, Step: 3070, Loss: 0.000886933
Epoch: 28, Step: 3080, Loss: 0.00126515
Epoch: 28, Step: 3090, Loss: 0.000482632
Epoch: 28, Step: 3100, Loss: 0.00178859
Epoch: 28, Step: 3110, Loss: 0.000344491
Epoch: 28, Step: 3120, Loss: 0.000421072
Epoch: 28, Step: 3130, Loss: 0.0016201
Epoch: 28, Step: 3140, Loss: 0.00103422
Epoch: 28, Step: 3150, Loss: 0.00177448
Epo

## Testing

In [22]:
tf.reset_default_graph()

In [6]:
sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, "save\\model.ckpt")

img = cv2.imread('steering_wheel_image.jpg',0)
rows,cols = img.shape

smoothed_angle = 0

i = 0
while(cv2.waitKey(10) != ord('q')):
    full_image = scipy.misc.imread("driving_dataset\\driving_dataset\\" + str(i) + ".jpg", mode="RGB")
    image = scipy.misc.imresize(full_image[-150:], [66, 200]) / 255.0
    degrees = y.eval(feed_dict={x: [image], keep_prob: 1.0})[0][0] * 180.0 / scipy.pi
    call("clear", shell=True)
    print("Predicted steering angle: " + str(degrees) + " degrees")
    cv2.imshow("frame", cv2.cvtColor(full_image, cv2.COLOR_RGB2BGR))
    #make smooth angle transitions by turning the steering wheel based on the difference of the current angle
    #and the predicted angle
    smoothed_angle += 0.2 * pow(abs((degrees - smoothed_angle)), 2.0 / 3.0) * \
                                    (degrees - smoothed_angle) / abs(degrees - smoothed_angle)
    M = cv2.getRotationMatrix2D((cols/2,rows/2),-smoothed_angle,1)
    dst = cv2.warpAffine(img,M,(cols,rows))
    cv2.imshow("steering wheel", dst)
    i += 1

cv2.destroyAllWindows()

INFO:tensorflow:Restoring parameters from save\model.ckpt


C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\client\session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  if sys.path[0] == '':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  del sys.path[0]


Predicted steering angle: -1.4200599345822966 degrees
Predicted steering angle: -1.2676180097138399 degrees
Predicted steering angle: -1.2641418703131266 degrees
Predicted steering angle: -1.0951877496827525 degrees
Predicted steering angle: -1.0417214557467156 degrees
Predicted steering angle: -1.3245224501233317 degrees
Predicted steering angle: -1.3000323799674278 degrees
Predicted steering angle: -1.3937935883553665 degrees
Predicted steering angle: -1.2082950420079288 degrees
Predicted steering angle: -0.8667774726576519 degrees
Predicted steering angle: -0.871953048501229 degrees
Predicted steering angle: -1.171807531131486 degrees
Predicted steering angle: -1.6770223233287544 degrees
Predicted steering angle: -1.6134710416397824 degrees
Predicted steering angle: -1.6755427335998005 degrees
Predicted steering angle: -1.854823673368411 degrees
Predicted steering angle: -2.0028757075912536 degrees
Predicted steering angle: -1.9277487493629908 degrees
Predicted steering angle: -1.27

Predicted steering angle: -4.5510378600231824 degrees
Predicted steering angle: -6.768867304757554 degrees
Predicted steering angle: -6.359645912146348 degrees
Predicted steering angle: -8.196192853069451 degrees
Predicted steering angle: -8.888509365078406 degrees
Predicted steering angle: -5.10614526311669 degrees
Predicted steering angle: -3.818620026736613 degrees
Predicted steering angle: -2.63459819637844 degrees
Predicted steering angle: -1.9997935847282853 degrees
Predicted steering angle: -2.6964630616726537 degrees
Predicted steering angle: -1.65955454141327 degrees
Predicted steering angle: -1.3340968812358087 degrees
Predicted steering angle: -1.5047700751029105 degrees
Predicted steering angle: -1.7074405707974063 degrees
Predicted steering angle: -1.7021719336263876 degrees
Predicted steering angle: -1.5548746353068315 degrees
Predicted steering angle: -1.2164120815062542 degrees
Predicted steering angle: -1.526928062544078 degrees
Predicted steering angle: -0.97257667601

Predicted steering angle: 5.797115800972329 degrees
Predicted steering angle: 5.0752523175007225 degrees
Predicted steering angle: 3.552296863732653 degrees
Predicted steering angle: 3.0960022342485427 degrees
Predicted steering angle: 2.523512853490999 degrees
Predicted steering angle: 2.0884140085487313 degrees
Predicted steering angle: 1.978544225890141 degrees
Predicted steering angle: 1.7065150801648696 degrees
Predicted steering angle: 1.1794570228367829 degrees
Predicted steering angle: 0.8128057447230342 degrees
Predicted steering angle: 0.6881631166633525 degrees
Predicted steering angle: 0.6022991010598298 degrees
Predicted steering angle: 0.5075402581662463 degrees
Predicted steering angle: 0.4123578368224162 degrees
Predicted steering angle: 0.3102746832610519 degrees
Predicted steering angle: 0.3348415930451184 degrees
Predicted steering angle: 0.24859668767366158 degrees
Predicted steering angle: 0.21856712065746584 degrees
Predicted steering angle: 0.10006077723680586 de

...

Predicted steering angle: -0.46411946495200584 degrees
Predicted steering angle: -0.4728420434089352 degrees
Predicted steering angle: 0.08310643344783593 degrees
Predicted steering angle: 0.051468997212336866 degrees
Predicted steering angle: 0.2613031874074125 degrees
Predicted steering angle: -0.012023053929926508 degrees
Predicted steering angle: -0.14087798771680207 degrees
Predicted steering angle: -0.8771307587789219 degrees
Predicted steering angle: -1.2387564050457163 degrees
Predicted steering angle: -1.3262746070888516 degrees
Predicted steering angle: -1.8129298543204913 degrees
Predicted steering angle: -2.060467008899155 degrees
Predicted steering angle: -2.374087951244984 degrees
Predicted steering angle: -2.3872031951118777 degrees
Predicted steering angle: -2.1630201648527807 degrees
Predicted steering angle: -2.0700800732691245 degrees
Predicted steering angle: -1.47378662948044 degrees
Predicted steering angle: -1.0899001160481048 degrees
Predicted steering angle: -0

Predicted steering angle: 0.33395505583519186 degrees
Predicted steering angle: 0.36788434070234755 degrees
Predicted steering angle: 0.505484584669484 degrees
Predicted steering angle: 0.2532216862371123 degrees
Predicted steering angle: -0.06811021951654403 degrees
Predicted steering angle: -0.0600384300214699 degrees
Predicted steering angle: 0.8409969769962916 degrees
Predicted steering angle: 1.0403638489274698 degrees
Predicted steering angle: 1.0563897138761411 degrees
Predicted steering angle: 0.8714654370275137 degrees
Predicted steering angle: 0.8805623952283244 degrees
Predicted steering angle: 0.8615566468108234 degrees
Predicted steering angle: 0.9755256101884793 degrees
Predicted steering angle: 0.8513723011498054 degrees
Predicted steering angle: 0.6382734939748753 degrees
Predicted steering angle: 0.23713456302932237 degrees
Predicted steering angle: 0.6584323704233448 degrees
Predicted steering angle: 0.6197177909906547 degrees
Predicted steering angle: 0.6711931915053

Predicted steering angle: -6.893015384432644 degrees
Predicted steering angle: -7.10827477254112 degrees
Predicted steering angle: -7.417329306539972 degrees
Predicted steering angle: -7.861593838461574 degrees
Predicted steering angle: -7.298161288543467 degrees
Predicted steering angle: -6.542751330963343 degrees
Predicted steering angle: -7.403067871552985 degrees
Predicted steering angle: -8.543060594973964 degrees
Predicted steering angle: -8.093570968337302 degrees
Predicted steering angle: -11.024690301587622 degrees
Predicted steering angle: -10.734313346760992 degrees
Predicted steering angle: -11.135995064302149 degrees
Predicted steering angle: -10.213624120668825 degrees
Predicted steering angle: -10.263802105406324 degrees
Predicted steering angle: -8.0905195813256 degrees
Predicted steering angle: -7.158653394085487 degrees
Predicted steering angle: -6.4350571769988125 degrees
Predicted steering angle: -5.500554963625899 degrees
Predicted steering angle: -4.47058121980819

Predicted steering angle: 0.7285113919681451 degrees
Predicted steering angle: 0.831672434757902 degrees
Predicted steering angle: 0.8666687232394606 degrees
Predicted steering angle: 0.971236891719946 degrees
Predicted steering angle: 0.9513979669454092 degrees
Predicted steering angle: 0.9790491340265596 degrees
Predicted steering angle: 1.021676984966844 degrees
Predicted steering angle: 0.9829928212203015 degrees
Predicted steering angle: 1.0017753078293077 degrees
Predicted steering angle: 1.0302448169503902 degrees
Predicted steering angle: 1.082209002877553 degrees
Predicted steering angle: 1.0633673924435438 degrees
Predicted steering angle: 1.2316355059491548 degrees
Predicted steering angle: 1.1898802116753397 degrees
Predicted steering angle: 1.2264042213751598 degrees
Predicted steering angle: 1.2122702121048763 degrees
Predicted steering angle: 1.2114405575641314 degrees
Predicted steering angle: 1.1507966953057631 degrees
Predicted steering angle: 1.2123275216608809 degre

Predicted steering angle: 1.4496740339978853 degrees
Predicted steering angle: 1.7067606468098726 degrees
Predicted steering angle: 1.9029715978185349 degrees
Predicted steering angle: 2.0384498941096187 degrees
Predicted steering angle: 2.231791205171523 degrees
Predicted steering angle: 1.8040279833978492 degrees
Predicted steering angle: 1.881831735450522 degrees
Predicted steering angle: 1.8857023182758037 degrees
Predicted steering angle: 2.131085615388386 degrees
Predicted steering angle: 2.5848782612021557 degrees
Predicted steering angle: 2.9290288684774723 degrees
Predicted steering angle: 2.7263690449535547 degrees
Predicted steering angle: 3.042965175013159 degrees
Predicted steering angle: 2.5811242184795873 degrees
Predicted steering angle: 2.589200063399481 degrees
Predicted steering angle: 2.3597552261585353 degrees
Predicted steering angle: 2.282911115684249 degrees
Predicted steering angle: 2.072724210251566 degrees
Predicted steering angle: 2.1597719830156166 degrees
